<a href="https://colab.research.google.com/github/cetusian/NER-product-names/blob/main/Fine_tune_distillbert_NER_Furniture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets evaluate seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 7.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.6 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=e9783d2da263c2e8df870ba858943dc1f007d862e00923024a6a6a39f11d1bc0
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling req

In [28]:
from huggingface_hub  import notebook_login
notebook_login()

In [3]:
import pandas as pd
from transformers import DistilBertTokenizerFast
import tensorflow as tf
import datasets

In [35]:
import json

def load_json(json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)
    return data

# Replace 'train.json' and 'validation.json' with the paths to your JSON files
train_data = load_json('/content/train_data.json')
validation_data = load_json('/content/val_data.json')

In [5]:
# Rename 'tokens' to 'input_ids' in each example in the train dataset
for example in train_data:
  example['input_ids'] = example.pop('tokens')

# Rename 'tokens' to 'input_ids' in each example in the validation dataset
for example in validation_data:
  example['input_ids'] = example.pop('tokens')

In [6]:
# Print the first few examples from the datasets to verify the changes
print("Train Dataset:", train_data[:3])
print("Validation Dataset:", validation_data[:3])

Train Dataset: [{'id': '77_49', 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0], 'input_ids': ['THROWSBOOKS', 'COOKING', 'HEALTHBOOKS', 'PLANTS', 'HOMECUSHIONSFOLIAGE', 'PATTERN', 'CUSHIONSLINEN', 'NATURAL', 'CUSHIONSSTRIPE', 'CHECK', 'CUSHIONSVELVET', 'VELOUR', 'BOUCLEFEATHER', 'FILLED', 'CUSHION', 'INSERTSALL', 'CUSHIONSDOORMATSGAMES', 'FAMILY', 'FUNLANTERNSRUGS', 'RUNNERSTISSUE', 'BOX', 'COVERSGIFT', 'VOUCHERS', 'BASKETS', 'STORAGE', 'BLANKETS', 'THROWS', 'BOOKS', 'COOKING', 'HEALTH', 'BOOKS', 'PLANTS', 'HOME', 'CUSHIONSFOLIAGE', 'PATTERN', 'CUSHIONSLINEN', 'NATURAL', 'CUSHIONSSTRIPE', 'CHECK', 'CUSHIONSVELVET', 'VELOUR', 'BOUCLEFEATHER', 'FILLED', 'CUSHION', 'INSERTSALL', 'CUSHIONS', 'FOLIAGE', 'PATTERN', 'CUSHIONS', 'LINEN']}, {'id': '0_188', 'ner_tags': [1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 2, 1, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 

In [7]:
type(train_data)

list

In [8]:
from datasets import Dataset

# Convert data to Hugging Face dataset format
train_dataset = Dataset.from_dict({"id": [example['id'] for example in train_data],
                                   "ner_tags": [example['ner_tags'] for example in train_data],
                                   "input_ids": [example['input_ids'] for example in train_data]})

validation_dataset = Dataset.from_dict({"id": [example['id'] for example in validation_data],
                                        "ner_tags": [example['ner_tags'] for example in validation_data],
                                        "input_ids": [example['input_ids'] for example in validation_data]})

type(train_dataset)

datasets.arrow_dataset.Dataset

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [10]:
example = train_dataset[0]
tokenized_input = tokenizer(example["input_ids"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
example, tokens

({'id': '77_49',
  'ner_tags': [0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   1,
   0,
   0,
   1,
   0,
   0,
   1,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   0,
   1,
   0,
   0,
   1,
   0],
  'input_ids': ['THROWSBOOKS',
   'COOKING',
   'HEALTHBOOKS',
   'PLANTS',
   'HOMECUSHIONSFOLIAGE',
   'PATTERN',
   'CUSHIONSLINEN',
   'NATURAL',
   'CUSHIONSSTRIPE',
   'CHECK',
   'CUSHIONSVELVET',
   'VELOUR',
   'BOUCLEFEATHER',
   'FILLED',
   'CUSHION',
   'INSERTSALL',
   'CUSHIONSDOORMATSGAMES',
   'FAMILY',
   'FUNLANTERNSRUGS',
   'RUNNERSTISSUE',
   'BOX',
   'COVERSGIFT',
   'VOUCHERS',
   'BASKETS',
   'STORAGE',
   'BLANKETS',
   'THROWS',
   'BOOKS',
   'COOKING',
   'HEALTH',
   'BOOKS',
   'PLANTS',
   'HOME',
   'CUSHIONSFOLIAGE',
   'PATTERN',
   'CUSHIONSLINEN',
   'NATURAL',
   'CUSHIONSSTRIPE',
   'CHECK',
   'CUSHIONSVELVET',
   'VELOU

In [11]:
for i, label in enumerate(train_dataset[0][f"ner_tags"]):
  print(i, label)

0 0
1 0
2 0
3 0
4 0
5 0
6 0
7 0
8 0
9 0
10 0
11 0
12 0
13 0
14 0
15 0
16 0
17 0
18 0
19 0
20 0
21 0
22 0
23 1
24 1
25 0
26 0
27 1
28 0
29 0
30 1
31 0
32 0
33 0
34 0
35 0
36 0
37 0
38 0
39 0
40 0
41 0
42 0
43 0
44 0
45 1
46 0
47 0
48 1
49 0


In [12]:
def tokenize_and_align_labels(examples):
  tokenized_inputs = tokenizer(examples["input_ids"], truncation=True, is_split_into_words=True)

  labels = []
  for i, label in enumerate(examples[f"ner_tags"]):
    word_ids = tokenized_inputs.word_ids(batch_index=i)
    previous_word_idx = None
    label_ids = []
    for word_idx in word_ids:
      if word_idx is None:
        label_ids.append(-100)
      elif word_idx != previous_word_idx:
        label_ids.append(label[word_idx])
      else:
        label_ids.append(-100)
      previous_word_idx = word_idx
    labels.append(label_ids)

  tokenized_inputs["labels"] = labels
  return tokenized_inputs

In [13]:
tokenized_train = train_dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/1868 [00:00<?, ? examples/s]

In [14]:
tokenized_val = validation_dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/468 [00:00<?, ? examples/s]

In [15]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer, return_tensors="tf")

In [16]:
import evaluate

seqeval = evaluate.load("seqeval")

In [17]:
label_list = [
    "O",
    "B-product",
    "I-product",
]

In [18]:
labels = [label_list[i] for i in example[f"ner_tags"]]
labels

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-product',
 'B-product',
 'O',
 'O',
 'B-product',
 'O',
 'O',
 'B-product',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-product',
 'O',
 'O',
 'B-product',
 'O']

In [19]:
import numpy as np

labels = [label_list[i] for i in example[f"ner_tags"]]

In [20]:
from seqeval.metrics import classification_report, f1_score, precision_score, recall_score, accuracy_score

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[pred] for (pred, lab) in zip(prediction, label) if lab != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[lab] for (pred, lab) in zip(prediction, label) if lab != -100]
        for prediction, label in zip(predictions, labels)
    ]

    precision = precision_score(true_labels, true_predictions, zero_division=0)
    recall = recall_score(true_labels, true_predictions, zero_division=0)
    f1 = f1_score(true_labels, true_predictions, zero_division=0)
    accuracy = accuracy_score(true_labels, true_predictions)

    print(f"Sample true labels: {true_labels[:2]}")   # Print sample true labels
    print(f"Sample predictions: {true_predictions[:2]}")  # Print sample predictions

    return {
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "accuracy": accuracy,
    }

In [21]:
id2label = {
    0: "O",
    1: "B-product",
    2: "I-product",
}

label2id = {
    "O": 0,
    "B-product": 1,
    "I-product": 2,
}

In [22]:
from transformers import create_optimizer

batch_size = 16
num_train_epochs = 3
num_train_steps = (len(tokenized_train) // batch_size) * num_train_epochs

optimizer, lr_schedule = create_optimizer(
    init_lr=3e-5,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
    num_warmup_steps=0,
)

In [23]:
from transformers import TFAutoModelForTokenClassification

model = TFAutoModelForTokenClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=3, id2label=id2label, label2id=label2id
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForTokenClassification: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForTokenClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForTokenClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForTokenClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able t

In [24]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_train,
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    tokenized_val,
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [25]:
import tensorflow as tf

model.compile(optimizer=optimizer)

In [26]:
from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)

In [29]:
from transformers.keras_callbacks import PushToHubCallback

push_to_hub_callback = PushToHubCallback(
    output_dir="ner-model-furniture-v2",
    tokenizer=tokenizer,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'Repository' (from 'huggingface_hub.repository') is deprecated and will be removed from version '1.0'. Please prefer the http-based alternatives instead. Given its large adoption in legacy code, the complete removal is only planned on next major release.
For more details, please read https://huggingface.co/docs/huggingface_hub/concepts/git_vs_http.
  warnings.warn(warning_message, FutureWarning)
Cloning https://huggingface.co/cetusian/ner-model-furniture-v2 into local empty directory.


In [30]:
callbacks = [metric_callback, push_to_hub_callback]

In [31]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=5, callbacks=callbacks)

Epoch 1/5


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
116/116 [==============================] - ETA: 0s - loss: 0.6457Sample true labels: [['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-product', 'B-product', 'O', 'O', 'O', 'O', 'O', 'O', 'B-product', 'B-product', 'B-product', 'B-product', 'O', 'B-product', 'B-product', 'O', 'O', 'B-product', 'O', 'O', 'O', 'O', 'B-product', 'O', 'O', 'B-product', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-product', 'O']]
Sample predictions: [['O', 'O', 'B-product', 'O', 'O', 'B-product', 'B-product', 'O', 'O', 'O', 'I-product', 'O', 'B-product', 'O', 'O', 'O', 'O', 'O', 'B-product', 'I-product'

In [32]:
text = "Albert : Sofa The Albert sofa is a luxurious piece of furniture featuring removable seat, back and side cushions with timber legs."

In [33]:
from transformers import pipeline

classifier = pipeline("ner", model="cetusian/ner-model-furniture-v2")

config.json:   0%|          | 0.00/696 [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/266M [00:00<?, ?B/s]

Some layers from the model checkpoint at cetusian/ner-model-furniture-v2 were not used when initializing TFDistilBertForTokenClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForTokenClassification were not initialized from the model checkpoint at cetusian/ner-model-furniture-v2 and are newly initialized: ['dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [34]:
classifier(text)

[{'entity': 'B-product',
  'score': 0.8928859,
  'index': 3,
  'word': 'sofa',
  'start': 9,
  'end': 13},
 {'entity': 'B-product',
  'score': 0.8876294,
  'index': 6,
  'word': 'sofa',
  'start': 25,
  'end': 29},
 {'entity': 'B-product',
  'score': 0.7796104,
  'index': 21,
  'word': 'cushions',
  'start': 104,
  'end': 112}]